# Review of Neural Network Optimization

Hello there! Our main objectives for this exercise are threefold:

Firstly, we aim to review some basic concepts of neural network training and establish a common vocabulary. It's always a good idea to start with the fundamentals, and we promise to make it fun and engaging for you!

Secondly, we'll work together to illustrate the basic principles involved in training a neural network, with specific focus on stochastic gradient descent. We'll take you through the process step-by-step, so you'll have a clear understanding by the end.

Lastly, this lab will create a solid foundation for the upcoming labs in this class, eventually leading us to a multi-GPU implementation of a neural network. We're excited to be taking you on this journey with us!

**Bonus point:** To reach these goals, we'll use some TensorFlow code (which may be low-level) that will help you improve your coding skills.

We'll begin with the simplest possible example: linear regression using a single input variable. You can model this as the simplest neural network you can imagine: a single neuron (i.e., the activation function is the linear function.

Here it comes a picture of the network

## Gradient descent

Gradient Descent (GD) is an optimization algorithm commonly used for finding the minimum of a function. It is widely used in ML for optimizing the parameters of a model (e.g., neural network) to minimize a cost or loss function.

Basic idea: iteratively update the parameters of the model in the opposite direction of the gradient of the cost function. 

High-level overview:

1. The algorithm starts by initializing the model's parameters with some initial values.
2. Compute the gradient: In each iteration, GD calculates the gradient of the cost function with respect to the parameters. The gradient is a vector that indicates the direction of the steepest ascent of the cost function. In neural networks this step is normally done by a technique called back-propagation. 
3. Update the parameters: GD updates the parameters by taking a step in the opposite direction of the gradient. The size of the step is determined by the learning rate (typically a small positive value), which controls the magnitude of the update. 
4. Repeat steps 2 and 3: The process of computing the gradient and updating the parameters is repeated iteratively until a stopping criterion is met. This criterion can be a maximum number of iterations, reaching a certain level of convergence, or other conditions specific to the problem.

In the description avobe we use “minimise the cost or loss function”. This loss function is computed by considering the prediction and actual labels of **the whole** training set. 

GD can converge to a local minimum of the cost function, which may not be the global minimum in the case of non-convex functions. Various techniques, such as momentum, adaptive learning rates, and regularization, can be employed to improve the convergence and avoid getting stuck in poor local minima.

In the next, we are going to implement this algorithm using tensorflow. Let start by defining our dataset.

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.models        import Sequential
from tensorflow.keras.layers        import Dense
from tensorflow.keras.optimizers    import SGD
from tensorflow.keras.losses        import mse


x_train = np.linspace(-1,1,101)
y_train = 5.0 * x_train + (np.random.randn(*x_train.shape) *.33 + 4.0)


# plot the line
plt.scatter(x_train,y_train)
plt.xlabel('b')
plt.ylabel('w')
plt.show()

So 'x_train' and 'y_train' are going to be the dataset values we are considering for training our model. We are not going to use a validation dataset in this notebook for the time being and we will visually validate the obtained results. 

Next we are going to define the tensorflow/keras model that implements the linear regression model above. In particular that model consists of an input layer that takes the variable x (actually a batch of it). That input layer is passed onto a Dense layer consisting of a single neuron. That neuron has a weight to be set `w` and a bias `b` to be determined (exactly as the figure above). We will rely on the Keras sequential model for this and will add a Dense layer. See the code cell below. When no activation is indicated `None` is used (which is equivalent to use a linear activation.)

In [ ]:
model = Sequential()
model.add(Dense(1,input_shape=(1,)))

The next thing that is needed is a loss function that can capture the quality of our model w.r.t. the dataset. Remember from the discussion above, the whole dataset is used for computing the loss in the gradient descent algorithm.

In this notebook, we will consider the the mean squared error loss function. 

The mean squared error loss function is a commonly used objective function in regression problems. It measures the average squared difference between the predicted values and the actual values of our target problem. 

The MSE loss function is widely used in regression tasks because it penalizes larger deviations between predicted and true values more than smaller deviations due to the squaring operation. It provides a continuous, differentiable, and non-negative measure of the model's performance, making it suitable for optimization using various algorithms.

This function is already provided within the set of losses Keras implements. 

In [ ]:
loss_function = tf.keras.losses.MeanSquaredError()

In keras there is no pure gradient descent algorithm. Instead, keras provides stochastic gradient descent. The stochastic gradient descent algorithm is equivalent to the gradient descent algorithm when the batch size is the whole dataset. 

## Stochastic gradient descent vs gradient descent

1. SGD performs the gradient update for each training sample individually or in small subsets (mini-batches) of the dataset.
2. Gradient computations involve thus less elements (single elements or mini-batch) thus resulting on faster computations
3. Since SGD computes the gradient based on a single example or mini-batch, the gradient estimate is noisier compared to GD. This noise introduces variance but can help SGD to escape shallow local minima and potentially converge to a better solution. We will see later in this notebook what this means in terms of the final obtained results. 
4. GD tends to converge more slowly compared to SGD, especially when dealing with large datasets. SGD's noisy updates allow it to converge faster initially, but it may oscillate around the optimal solution due to the noise in gradient estimation. Techniques such as learning rate scheduling and momentum can be employed in SGD to alleviate this issue.
5. GD requires memory to store the entire dataset, which can be a limitation for large datasets. In contrast, SGD only requires memory to store a single training example or mini-batch.
6. GD calculates the slope using the entire set of data, which can result in a more precise estimate of the actual slope. Therefore, it may perform better on new data (this assumes the whole dataset might be a better representation of these new data). However, SGD's updates that include noise can introduce some regularization, which can help avoid overfitting and improve generalization in specific scenarios.

In [ ]:
loss_function = tf.keras.losses.MeanSquaredError()
model.compile(optimizer=SGD(),loss=loss_function)
model.summary()

Now we have our model and would like to start training it. We would also like to see how the parameters change after every update. For doing this, we are going to write a `Callback` class in tensorflow. Callbacks are a mechanism that allow us to customise the behaviour of tensorflow during training or inference at determined points. These determined points are for example: before the training begins, before a new epoch starts being processed, after the training ends, after every training batch has been processed, etc. An overview of different defined points in Keras can be found here:
https://www.tensorflow.org/guide/keras/writing_your_own_callbacks

In our case, we are going to overwrite the `on_train_batch_end` method of the call back as we are interested in knowing the value of these parameters every time they have been upgraded by the optimization loop. 


In [ ]:
# Our custom callback only records the values of the parameters after 
# every epoch. We can later gather that information out of a callback 
# object

class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        self.w      = []
        self.b      = []
        self.loss   = []

    def on_train_batch_end(self, batch, logs=None):
        self.w.append(self.model.layers[0].weights[0].numpy()[0][0])
        self.b.append(self.model.layers[0].weights[1].numpy()[0])
        self.loss.append(logs['loss'])


In [ ]:
callback = CustomCallback()
history = model.fit(x_train, y_train,batch_size=len(x_train), epochs=200,callbacks=[callback])

Ok, now we have trained our model. Let us visually check how well it has performed for our task. 

In [ ]:
plt.plot(history.history['loss'],label='loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()


In [ ]:

y_model = model.predict(x_train)
plt.plot(x_train,y_model,color='red',linewidth=2,label='Line of best Fit')
plt.scatter(x_train,y_train,label='Test data plots')
plt.legend()
plt.show()

Ok, we can visually check that our model has found a more or less accurate line that represent our points. 

We can indeed print the loss function with respect to the parameters governing our model and see how the gradient descent navigated that search space. 

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
loss_surface        = []
w_surface           = []
b_surface           = []


def model_function(X,w,b):
    return X * w + b


for w_value in np.linspace(0, 10, 200):
    for b_value in np.linspace(0, 10, 200):
        y_model         = model_function(x_train,w_value,b_value)

        loss        = loss_function(y_model,y_train)

        b_surface.append(b_value)
        w_surface.append(w_value)
        loss_surface.append(loss)

plt.close()

fig = plt.figure()
ax2 = fig.add_subplot(projection='3d')

ax2.scatter(w_surface, b_surface, loss_surface,         c = loss_surface,       alpha = 0.02)
ax2.plot(callback.w,callback.b,callback.loss, color='black')

ax2.set_xlabel('w')
ax2.set_ylabel('b')

plt.show()

In [ ]:
!pip3 install ipympl

The gradients are computed using that loss function and therefore the weights update happens by considering it. In the following picture we are plotting both the loss function with respect to all the dataset and the loss function with respect to a single random batch

In [ ]:
loss_surface        = []
loss_batch_surface  = []
w_surface           = []
b_surface           = []


indexes = np.random.choice(len(x_train),1)

x_batch = np.take(x_train,indexes)
y_batch = np.take(y_train,indexes)

def model_function(X,w,b):
    return X * w + b


for w_value in np.linspace(0, 10, 200):
    for b_value in np.linspace(0, 10, 200):
        y_model         = model_function(x_train,w_value,b_value)
        y_model_batch   = model_function(x_batch,w_value,b_value)


        loss        = loss_function(y_model,y_train)
        loss_batch  = loss_function(y_batch,y_model_batch)

        b_surface.append(b_value)
        w_surface.append(w_value)
        loss_surface.append(loss)
        loss_batch_surface.append(loss_batch)

plt.close()

fig = plt.figure()
ax2 = fig.add_subplot(projection='3d')

ax2.scatter(w_surface, b_surface, loss_surface,         c = loss_surface,       alpha = 0.02)
ax2.scatter(w_surface, b_surface, loss_batch_surface ,  c = loss_batch_surface, alpha = 0.5)
ax2.plot(callback.w,callback.b,callback.loss, color='black')

ax2.set_xlabel('w')
ax2.set_ylabel('b')

plt.show()

As we can see there are notable differences between the losses in both cases. It is possible to see that points of both surfaces have different gradients (e.g., can you see these differences on the point w=0.0 and b=0.0 in the two computed surfaces). In fact, for a given point the gradient in both surfaces might point to different directions. Think that when using a mini-batch, the gradient is optimizing for that mini-batch and that every step you have a different loss function. This is what normally many people refer to when saying it introduces noise that migh help to scape from local optima, better navigate plateus of the loss function, etc. 

Ok, in the following we are going to experiment with a smaller batch size and the stochastic gradient descent and compare the results with the ones above. 

In [ ]:
model = Sequential()
model.add(Dense(1, input_shape=(1,)))
loss_function = tf.keras.losses.MeanSquaredError()
model.compile(optimizer=SGD(),loss=loss_function)
model.summary()
y_model = model.predict(x_train)
initial_loss = loss_function(y_train,y_model)
callback = CustomCallback()
history = model.fit(x_train, y_train,batch_size=16, epochs=200,callbacks=[callback])

In [ ]:
plt.close()
y_model = model.predict(x_train)
plt.plot(x_train,y_model,color='red',linewidth=2,label='Line of best Fit')
plt.scatter(x_train,y_train,label='Test data plots')
plt.legend()
plt.show()

Visually, we can observe that the model now converged much better to a line that approximate all the points as best as possible. Lets show again the loss function and the different values of the parameters that were chosen by the stochastic gradient descent algorithm. 

In [ ]:
loss_surface        = []
w_surface           = []
b_surface           = []


for w_value in np.linspace(0, 10, 200):
    for b_value in np.linspace(0, 10, 200):
        y_model         = model_function(x_train,w_value,b_value)

        loss        = loss_function(y_model,y_train)

        b_surface.append(b_value)
        w_surface.append(w_value)
        loss_surface.append(loss)

plt.close()

fig = plt.figure()
ax2 = fig.add_subplot(projection='3d')

ax2.scatter(w_surface, b_surface, loss_surface,         c = loss_surface,       alpha = 0.02)
ax2.plot(callback.w,callback.b,callback.loss, color='black')

ax2.set_xlabel('w')
ax2.set_ylabel('b')

plt.show()